# Load and set data

In [2]:
#load library and set config
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option("display.precision", 9)
date_parser = pd.to_datetime

In [4]:

df_rentabilidad = pd.read_csv('../01data/01origin/RentabilidadProduct.csv',
                                sep='|',
                                encoding='utf-8',
                                dtype={'product_id': 'str',
                                        'family_id': 'str',
                                        'negocio_id': 'str',
                                        'category_id': 'str',
                                        'tier_id': 'str',
                                        'utilidad_bruta_transf': np.float64,})

df_train = pd.read_csv('../01data/01origin/train.csv',
                        sep='|',
                        encoding='utf-8',
                        dtype={'fecha_compra':'str',
                                'customer_id':'str',
                                'product_id':'str',
                                'type_id':'str',
                                'business_id':'str',
                                'channel_id':'str',
                                'cantidad_venta':np.float64,
                                'monto_venta_transf':np.float64,
                                'peso_venta_transf':np.float64,})

df_train['fecha_compra'] = pd.to_datetime(df_train['fecha_compra'], format='%Y-%m-%d')

df_submit = pd.read_csv('../01data/01origin/submit_example.csv')

In [31]:
df_rentabilidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             937 non-null    object 
 1   family_id              937 non-null    object 
 2   negocio_id             937 non-null    object 
 3   category_id            937 non-null    object 
 4   tier_id                937 non-null    object 
 5   utilidad_bruta_transf  937 non-null    float64
dtypes: float64(1), object(5)
memory usage: 44.0+ KB


In [32]:
df_rentabilidad.head()

,product_id,family_id,negocio_id,category_id,tier_id,utilidad_bruta_transf
0,881,1291,20,131,4,10.378016570
1,805,1224,20,131,3,10.177925190
2,892,1089,20,146,1,10.008242890
3,841,1224,20,131,3,7.698361552
4,830,1089,20,131,1,7.353677620


In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167848 entries, 0 to 2167847
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   fecha_compra        datetime64[ns]
 1   customer_id         object        
 2   product_id          object        
 3   type_id             object        
 4   business_id         object        
 5   channel_id          object        
 6   cantidad_venta      float64       
 7   monto_venta_transf  float64       
 8   peso_venta_transf   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 148.9+ MB


In [34]:
df_train.head()

,fecha_compra,customer_id,product_id,type_id,business_id,channel_id,cantidad_venta,monto_venta_transf,peso_venta_transf
0,2022-01-31,7157,510,BO,21,V,1.0,0.153430522,0.005210848
1,2022-01-31,7157,512,BO,21,V,1.0,0.153430522,0.005210848
2,2022-01-31,9382,510,BO,21,V,1.0,0.153430522,0.005210848
3,2022-01-31,9382,512,BO,21,V,1.0,0.153430522,0.005210848
4,2022-01-31,9382,525,BO,21,V,1.0,0.153430522,0.005789831


In [35]:
df_submit.head()

,customer_id,product_id
0,1,10 15 70 30 77 890 194 88 915 926 1028 190 452...
1,2,5 9 70 456 414 404 452 194 119 190 890 406 116...
2,3,80 2 7 452 456 116 404 414 77 890 194 406 926 ...
3,4,600 400 1 12 404 414 406 203 116 77 459 890 42...
4,5,70 60 20 56 119 81 328 972 495 174 813 37 801 ...


## Set real test for Kaggle

In [37]:
# create function to save dateframe to parquet
def save_parquet(df, path, filename):
    df.to_parquet(path + filename + '.parquet', engine='pyarrow', compression='snappy')
    return
#apply function save_parquet for 3 df
save_parquet(df_rentabilidad, '../01data/02preprocessed/01parquet/', 'df_rentabilidad')
save_parquet(df_train, '../01data/02preprocessed/01parquet/', 'df_train')
save_parquet(df_submit, '../01data/02preprocessed/01parquet/', 'df_submit')

In [6]:
# create function to save dateframe to csv and apply in all dataframe
def save_csv(df, path, filename):
    df.to_csv(path + filename + '.csv', sep='|', index=False)
    return
#apply function save_csv for 3 df
save_csv(df_rentabilidad, '../01data/02preprocessed/', 'df_rentabilidad')
save_csv(df_train, '../01data/02preprocessed/', 'df_train')
save_csv(df_submit, '../01data/02preprocessed/', 'df_submit')


In [7]:
#create a function to get a sample for all columns of dataframe df_train estratified by fecha_compra and product_id and save to csv
def get_sample(df, path, filename, n):
    df_sample = df.groupby(['fecha_compra', 'product_id'], group_keys=False).apply(lambda x: x.sample(min(len(x), n)))
    df_sample.to_csv(path + filename + '.csv', sep='|', index=False)
    return df_sample
#apply function get_sample for df_train
df_sample = get_sample(df_train, '../01data/02preprocessed/', 'df_train_sample', 100000)


In [31]:
def stratified_sample_updated(df: pd.DataFrame, stratify_column: str, sample_size: int) -> pd.DataFrame:
    """
    Function to get a stratified sample from a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame from which to take the sample.
    stratify_column (str): Column to use for stratification.
    sample_size (int): Size of the sample to take.

    Returns:
    sample_df (pd.DataFrame): DataFrame containing the stratified sample.
    """
    # Group the data by the stratify column and take a sample from each group
    groups = df.groupby(stratify_column)
    sample_df = pd.concat([
        group.sample(min(len(group), sample_size), random_state=42) 
        for _, group in groups
    ])

    return sample_df

sample_df_train = stratified_sample_updated(df_train, "product_id", 200)


In [33]:
print(sample_df_train.shape)
save_csv(sample_df_train, '../01data/02preprocessed/', 'df_train_sample')

(142526, 9)
